In [ ]:
#imports
import numpy as np
import numpy.random as rd
import math
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from hmmlearn import hmm

In [ ]:
def sample(prob): # BS can be done
    r = rd.random();i = 0
    while r > prob[i]:
        r-=prob[i]
        i+=1
    return i

def bernoulli(prob):
    if rd.random() > prob:
        return 1
    return 0

def row_normalize(arr):
    arr = np.array(arr,float)
    for i in range(np.shape(arr)[0]):
        arr[i] = arr[i]/np.sum(arr[i])
    return arr

def steady_state_prob(matrix):
    """
    This function calculates relative throughput from a probability matrix
    :param matrix: the probability matrix
    :return: A list of relative throughput values
    """
    a_list = []
    b_list = []
    for v in matrix:
        a_list.append(1)
        b_list.append(0)
    b_list.append(1)
    A = np.append(np.transpose(matrix) - np.identity(len(matrix)), [a_list], axis=0)
    b = np.transpose(np.array(b_list))
    return np.linalg.solve(np.transpose(A).dot(A), np.transpose(A).dot(b))

In [ ]:
en = 3
ch = 3
P = np.array([[.1, .2, .7],[.5, .3, .2],[.3, .3, .4]])
E = np.array([[.5,.6,.4],[.2,.3,.1],[.4,.5,.3]])
rate = np.array([8.,6.,4.])
# E = np.array([[.6,.9,.3],[.3,.5,.1],[.4,.1,.05]])
# P = row_normalize(np.random.rand(ch,ch)/2)
# E = np.random.rand(en,ch)/2
# rate = np.array([np.random.randint(1,10) for i in range(en)])
lr = 0.01
eps = .5
eps_min = 0.01
print(P)
print(E)
ub = np.sum(E*steady_state_prob(P),axis=1)
print(min(ub))

In [ ]:
tol = 0.25
ltemp = linprog(c = -1*rate, A_ub = [ub], b_ub = [tol],A_eq = np.ones((1,en)),b_eq = [1.0],method='simplex')
print(np.dot(ltemp.x,rate), np.sum(ltemp.x),ltemp.con, ltemp.success, np.dot(ltemp.x,ub))
print(ltemp.x)

In [ ]:
print(en,ch,tol,np.dot(ltemp.x,rate),min(ub))
print(P)
print(E)
print(ltemp.x)

In [ ]:
lamda = [np.ones(en)/en]
chnl = []; b = [] ; enc = []
emp_rate = [0]; emp_err = [0]
chnl.append(rd.randint(0,ch-1))
enc.append(rd.randint(0,en-1))
b.append(0)
t = 0; conf = 0.1; reg = 0.1
alfa_r = 1; alfa_c = 1

In [ ]:
np.identity(en) - np.linalg.norm(np.transpose([lamda[-1]])*lamda[-1])

In [ ]:
"""
x -> lamda
r -> <lamda,rate> - scalar?
c -> ?
"""
v = np.transpose([lamda[0]])*lamda[0]
d = np.linalg.norm(v)
Iv = np.identity(en) - v/(d*d)
I = np.identity(en)
cTo = [0]
T = 1000
while t < T:
    cTo.append(c[t] - (np.dot(x[t],c[0])/np.linalg.norm(x[0]))*c[0])
    sigT = reg*I
    for i in range(t+1):
        sigT += (x[i]*np.transpose(x[i]))
    sigTo = reg*Iv
    for i in range(t+1):
        sigTo += (Iv*x[i]*np.transpose(Iv*x[i]))
    tetaT = np.zeros(en)
    for i in range(t+1):
        tetaT += r[i]*x[i]
    tetaT = np.linalg.inv(sigT)*tetaT
    muTo = np.zeros((en,ch))
    for i in range(t+1):
        muTo += (cTo[i])*(Iv*x[i])
    muTo = np.linalg.inv(sigTo)*muTo
    t+=1

In [ ]:
# print(p[0])
print('estimated trans\n',p[-1])
print('real trans\n',P)
print('lamda\n',lamda[-1])
# print(sprob)
# print(model.transmat_)

In [ ]:
stat = 1
error_plot = 1.0 - np.copy(b)
for i in range(1,len(error_plot)):
    error_plot[i] += (error_plot[i-1]*(i-1))
    error_plot[i] =error_plot[i]/i
plt.plot(error_plot[stat:])
plt.title('empirical error:no. of 1s obtained over interval')
plt.show()
rate_plot = np.copy(emp_rate)
for i in range(1,len(rate_plot)):
    rate_plot[i] += (rate_plot[i-1]*(i-1))
    rate_plot[i] =rate_plot[i]/i
plt.plot(rate_plot[stat:])
plt.title('average rate:sum[over time](R*lamda)/t')
plt.show()

In [ ]:
print(rate_plot[-1],error_plot[-1])

In [ ]:
err = []; s = np.shape(P)[0]*np.shape(P)[1]
for i in p:
    err.append(np.sqrt(np.sum(np.square(P-i))/s)*100)
print(err[-1])
plt.plot(err)
plt.title('estimation error on transition matrix')
plt.show()

In [ ]:
rate_plot = [0]
for i in range(1,len(enc)):
    rate_plot.append((rate_plot[i-1]*(i-1))+rate[enc[i]])
    rate_plot[i] =rate_plot[i]/i
print(rate_plot[-1])
plt.plot(rate_plot[stat:])
plt.title('rate based on selected channels at each time (ignoring errors)')
plt.show()

In [ ]:
err = []; s = np.shape(P)[0]*np.shape(P)[1]
for i in p:
    err.append(np.sqrt(np.sum(np.square(steady_state_prob(P)-steady_state_prob(i)))/3)*100)
print(steady_state_prob(p[-1]))
print(steady_state_prob(P))
print(err[-1])
plt.plot(err)
plt.title('steady estimation error on transition matrix')
plt.show() 

In [ ]:
T = 10000
chnl = [0];enc = [1];b = [0]
t = 0
while t < T:
    enc.append(1)
    chnl.append(sample(P[chnl[-1]]))
    b.append(bernoulli(E[enc[-1]][chnl[-1]]))
    t+=1

In [ ]:
# reference
model = hmm.MultinomialHMM(n_components=3, algorithm='viterbi', params='t', init_params='t',n_iter=100, tol=1.0e-2)
model.emissionprob_ = [[E[1][0],1.0-E[1][0]],[E[1][1],1.0-E[1][1]],[E[1][2],1.0-E[1][2]]]
model.startprob_ = np.ones(ch)/ch
model.fit([b])
# print(model.emissionprob_,model.startprob_)
print(model.transmat_,P)
s = np.shape(P)[0]*np.shape(P)[1]
print('Error',np.sqrt(np.sum(np.square(P-model.transmat_))/s)*100)